In [1]:
import os
os.environ['HF_HOME'] = '/mnt/sagemaker-nvme/cache'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import torch
import pandas as pd
import json
from tqdm import trange, tqdm
from sklearn.cluster import KMeans
from utils import last_token_pool, get_detailed_instruct, get_llm_response, PersonaDimension, list_s3_prefix
from transformers import AutoTokenizer, AutoModel
from dataclasses import asdict

surveys = set()
for path in list_s3_prefix("human_resp/"):
    if path.startswith("human_resp/American_Trends_Panel"):
        # Extract the folder name
        folder = path.split("/")[1]
        surveys.add(folder)
surveys = sorted(list(surveys))[:3]
surveys

['American_Trends_Panel_W26',
 'American_Trends_Panel_W27',
 'American_Trends_Panel_W29']

# Clustering

In [2]:
def get_personas_extracted_from_questions(personas_from_questions_filename):
    with open(personas_from_questions_filename, 'r') as f:
        data = json.load(f)
    res = []
    for entry in data:
        if not entry['valid']: continue
        for persona_dim in eval(entry['response']):
            entry_dict = asdict(persona_dim)
            input_dict = eval(entry['input_dict'])
            entry_dict['original_question'] = input_dict['question'] + ' ' + input_dict['options']
            res.append(entry_dict)
    df = pd.DataFrame(res)
    return df


def cluster_extracted_personas(survey, extraction_dir, output_dir, num_clusters, print_result, tokenizer, model):
    os.makedirs(output_dir, exist_ok=True)
    
    # Get data
    personas_from_questions_filename = f"{extraction_dir}/personas_extracted_from_question_{survey}.json"
    data = get_personas_extracted_from_questions(personas_from_questions_filename)

    # Get formatted string for clustering
    def get_formatted_persona_dim(row):
        task = 'Given a persona dimension description, retrieve semantically similar persona dimension descriptions.'
        persona = f"{row['name']}: {row['description']}. Candidate values: {row['candidate_values']}"
        return get_detailed_instruct(task, persona)
    data['formatted'] = data.apply(get_formatted_persona_dim, axis=1)

    for level in ['high', 'mid', 'low']:
        # Get subset and save artifacts
        level_df = data[data['level']==level]
        level_df.to_csv(f"{output_dir}/{level}-level_personas_{survey}.csv")
        
        # Get the embeddings
        max_length = 4096
        input_texts = level_df['formatted'].to_list()

        embeddings = []
        for text in input_texts:
            batch_dict = tokenizer([text], max_length=max_length, padding=True, truncation=True, return_tensors="pt")
            with torch.no_grad():
                outputs = model(**batch_dict)
                embed = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])[0]
            embeddings.append(embed)
        embeddings = torch.stack(embeddings, axis=0)
        print(embeddings.shape, len(input_texts))
        
        # Clustering the embeddings and save artifacts
        clustering_model = KMeans(n_clusters=num_clusters)
        clustering_model.fit(embeddings)
        level_df['cluster'] = clustering_model.labels_
        level_df = level_df.sort_values(by='cluster')
        level_df.to_csv(f'{output_dir}/clustered_{level}_level_personas_{survey}.csv')

        if print_result:
            for idx in range(num_clusters):
                print(idx)
                for _, row in enumerate(level_df[level_df['cluster'] == idx]['formatted']):
                    print(row.split('\n')[1])
                print('\n\n')

In [3]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-2_R')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-2_R', device_map='auto')

for survey in tqdm(surveys):
    cluster_extracted_personas(survey,
                               extraction_dir='sm_local/outputs/extraction',
                               output_dir='sm_local/outputs/clustering',
                               num_clusters=20,
                               print_result=False,
                               tokenizer=tokenizer,
                               model=model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([226, 4096]) 226


/tmp/ipykernel_30235/4183643238.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level_df['cluster'] = clustering_model.labels_


torch.Size([226, 4096]) 226


...

torch.Size([205, 4096]) 205
torch.Size([223, 4096]) 223


/tmp/ipykernel_30235/4183643238.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level_df['cluster'] = clustering_model.labels_


torch.Size([305, 4096]) 305


/tmp/ipykernel_30235/4183643238.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level_df['cluster'] = clustering_model.labels_
/tmp/ipykernel_30235/4183643238.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level_df['cluster'] = clustering_model.labels_
100%|██████████| 3/3 [04:14<00:00, 84.77s/it]

torch.Size([158, 4096]) 158


# Summarizing

In [4]:
def summarize_clustered_personas(prompt_name, survey, level, clustering_dir, output_dir, num_clusters, print_result):
    os.makedirs(output_dir, exist_ok=True)

    # Get data
    clustered_persona_filename = f"{clustering_dir}/clustered_{level}_level_personas_{survey}.csv"
    data = pd.read_csv(clustered_persona_filename)
    with open(f'prompts/{prompt_name}.txt') as f:
        prompt_template = f.read()

    # summarize
    res = []
    for idx in trange(num_clusters):
        persona_cluster = []
        for _, row in data[data['cluster'].astype(str) == str(idx)].iterrows():
            persona = PersonaDimension(**row[['name', 'description', 'level', 'candidate_values']].to_dict())
            persona_cluster.append(persona)
        
        prompt = prompt_template.format(persona_dimensions='[\n' + ',\n'.join(repr(dim) for dim in persona_cluster) + '\n]')
        response = get_llm_response(prompt, prefill='[')
        response = '[' + response
        
        try:
            response = eval(response)
            for dim in response:
                res.append(dim)

            record_res = [str(dim) for dim in res]
            summarized_clustered_personas_filename = f"{output_dir}/summarized_{level}_level_personas_{survey}.json"
            with open(summarized_clustered_personas_filename, 'w') as f:
                json.dump(record_res, f, indent=4)
    
            if print_result:
                print(response)
        except:
            if print_result:
                print(f"cluster {idx} failed")
                print(response)
            continue


In [ ]:
for survey in surveys:
    for level in ['low', 'mid', 'high']:
        summarize_clustered_personas(prompt_name="summarize_clustered_personas",
                                     survey=survey,
                                     level=level,
                                     clustering_dir='sm_local/outputs/clustering',
                                     output_dir='sm_local/outputs/summarizing',
                                     num_clusters=20,
                                     print_result=True)

  5%|▌         | 1/20 [00:03<00:58,  3.07s/it]

[    PersonaDimension(
        name="Occupation and Gun Involvement",
        description="The participant's occupation, profession, or job responsibilities that may involve, require, or be influenced by gun ownership, usage, or exposure.",
        level="low",
        candidate_values=['gun-related occupation', 'non-gun-related occupation']
    )]


 10%|█         | 2/20 [00:06<00:59,  3.29s/it]

[    PersonaDimension(
        name="Geographic Gun Ownership and Risk",
        description="The participant's geographic location in terms of gun ownership rates, gun culture, and risk of gun-related incidents such as violence and crime.",
        level="low",
        candidate_values=['high gun ownership and risk area', 'low gun ownership and risk area']
    )]


 15%|█▌        | 3/20 [00:09<00:57,  3.37s/it]

...

[    PersonaDimension(
        name="Civic and Community Engagement",
        description="The individual's level of involvement, participation, and interest in local community organizations, political processes, social movements, and civic activities aimed at contributing to and improving their local community.",
        level="mid",
        candidate_values=['highly engaged', 'moderately engaged', 'disengaged']
    )]


 20%|██        | 4/20 [00:24<01:29,  5.61s/it]

[    PersonaDimension(
        name="Attitudes Towards Gender Norms and Roles",
        description="Beliefs, attitudes, and perceptions about societal expectations, norms, and roles related to gender, including traditional and progressive views on masculinity, femininity, and appropriate behaviors for men, women, boys, and girls.",
        level="mid",
        candidate_values=['traditional gender norms', 'progressive gender norms', 'egalitarian gender roles']
    )]


 25%|██▌       | 5/20 [00:27<01:07,  4.52s/it]

[    PersonaDimension(
        name="Adaptability and Tolerance for Ambiguity",
        description="The individual's ability and willingness to adapt to changes, embrace uncertainty, and adjust to ambiguous or unfamiliar situations in various contexts, such as the workforce, job market, and workplace dynamics.",
        level="mid",
        candidate_values=['highly adaptable', 'moderately adaptable', 'resistant to change']
    )]


# Cleaning

In [2]:
def clean_summarized_personas(prompt_name, survey, level, summarizing_dir, output_dir, num_clusters, print_result):
    os.makedirs(output_dir, exist_ok=True)

    # Get data
    summarized_persona_filename = f"{summarizing_dir}/summarized_{level}_level_personas_{survey}.json"
    with open(summarized_persona_filename, 'r') as f:
        data = json.load(f)
    with open(f'prompts/{prompt_name}.txt') as f:
        prompt_template = f.read()

    # summarize
    prompt = prompt_template.format(persona_dimensions='[\n' + ',\n'.join(repr(dim) for dim in data) + '\n]')
    response = get_llm_response(prompt, prefill='[', max_tokens=4096)
    response = '[' + response

    # validate
    try:
        eval(response)
        cleaned_summarized_personas_filename = f"{output_dir}/cleaned_{level}_level_personas_{survey}.json"
        with open(cleaned_summarized_personas_filename, 'w') as f:
            json.dump(response, f, indent=4)
        return response
    except:
        print(f'Cleaned result is not valid. Survey: {survey}, Level: {level}. Response:')
        # print(response)
        return None

In [ ]:
for survey in surveys:
    for level in tqdm(['low', 'mid', 'high']):
        cleaned_personas = clean_summarized_personas(prompt_name="clean_summarized_personas",
                                                      survey=survey,
                                                      level=level,
                                                      summarizing_dir='sm_local/outputs/summarizing',
                                                      output_dir='sm_local/outputs/cleaned',
                                                      num_clusters=20,
                                                      print_result=True)
        

In [5]:
for survey in surveys:
    for level in tqdm(['low', 'mid', 'high']):
        with open(f'sm_local/outputs/cleaned/cleaned_{level}_level_personas_{survey}.json') as f:
            json_string = json.load(f)
        response = eval(json_string)

        csv_res = []
        for entry in response:
            csv_res.append(asdict(entry))
        df = pd.DataFrame(csv_res)
        df.to_csv(f'sm_local/outputs/cleaned/cleaned_{level}_level_personas_{survey}.csv')

100%|██████████| 3/3 [00:00<00:00, 590.39it/s]
